## Extracción y almacenamiento de datos

### Justificación de elección de API:

En el presente trabajo decidi elegir la API JCDecaux Developers ya que la misma ofrece datos estáticos como la información fija de cada contrato particular, los cuales serán utiles para realizar una extracción full. Asimismo, ofrece datos dinámicos como la información en tiempo real de las estaciones de bicicletas de un contrato particular, por lo que resulta de gran utilidad para la extracción incremental. En este sentido, considero importante aclarar que para este trabajo utilice dos enpoints:
- Extracción full: Obtuve los metadatos de las estaciones de bicicletas.
- Extracción incremental: Utilice información en tiempo real sobre el estado de cada estación, la cual incluye las bicicletas disponibles, si se debe pagar, si se encuentra abierto, etc.

### Desarrollo del trabajo práctico con sus respectivos comentarios:

##### Instalación de librerias:

In [ ]:
!pip install requests
!pip install deltalake
!pip install pyarrow

##### Importación de librerias:

In [ ]:
import requests
import pandas as pd
import json
import pyarrow as pa
import math
from datetime import datetime, timezone
from deltalake import write_deltalake, DeltaTable
from deltalake.exceptions import TableNotFoundError

##### Definición de funciones:

In [ ]:
# Manejo de json

def get_last_update (file_path):

    try:
        #with es util porque permite simplificar el manejo de archivos, conexiones, db al asegurarse que se usan y liberan de forma correcta, incluso si hay errores.
        
        with open(file_path, "r") as file: 
            ultimoUpdate = json.load(file) # leo el archivo json
            return ultimoUpdate["last_updated"]

    except FileNotFoundError:
        raise FileNotFoundError(f"El archivo JSON en la ruta {file_path} no existe.") # propago el error y lo resuelvo en otro lado!
    
    except json.JSONDecodeError:
        raise json.JSONDecodeError(f"El archivo JSON en la ruta {file_path} no es válido.")

    except KeyError as k:
        raise KeyError(str(k))


def update_last_update (file_path, last_updated):

    try:
        with open(file_path, "w") as file:
            json.dump({"last_updated": last_updated.isoformat()}, file, indent=4)
    
    except FileNotFoundError:
        raise FileNotFoundError(f"El archivo JSON en la ruta {file_path} no existe.") # propago el error y lo resuelvo en otro lado!
    
# Extraccion de datos

def get_data_stations(url, params=None):
    
    try:
        response = requests.get(url,params=params, timeout = 10)
        response.raise_for_status() # excepcion que captura el except
        data = response.json()
        return create_data_frame(data)
        
    except requests.exceptions.RequestException as e:
        print(f"Error al obtener los datos. Código de error: {e}")
        return pd.DataFrame()

def create_data_frame(json_data):

    try:
        df = pd.json_normalize(json_data)
        return df
    
    except Exception as e:
        print(f"Se produjo un error en la construcción del DataFrame: {e}")   
        return pd.DataFrame() 
    
def incremental_extraction(url, file_path, params=None):

    try:

        ultimo_updateJson = get_last_update(file_path=file_path)
        ultimo_update = pd.to_datetime(ultimo_updateJson, utc=True)

        df = get_data_stations(url, params=params)
        if df is None:
            print("No se pudo construir el DataFrame.")
            return pd.DataFrame()
        
        fechas_convertidas = []
        for f in df["last_update"]:
            if f is not None and not math.isnan(f):
                ft = datetime.fromtimestamp(f / 1000, tz=timezone.utc)
                fechas_convertidas.append(ft)
            else:
                fechas_convertidas.append(pd.NaT) #fecha nula en caso de ser null
            
        df["last_update"] = fechas_convertidas

        df_incremental = df[df["last_update"] > ultimo_update]

        if df_incremental.empty:
            print("No hay nuevas actualizaciones desde la última consulta")
            return pd.DataFrame()

        max_timestamp = df["last_update"].max()
        update_last_update(file_path=file_path, last_updated=max_timestamp)
        return df_incremental

    except Exception as e:
        print(f"Se produjo un error en la extracción incremental {e}")
        return pd.DataFrame()


# Almacenamiento de datos

def save_data_as_delta(df, path, mode="overwrite", partition_cols=None):

    write_deltalake(path, df, mode = mode, partition_by = partition_cols)

def merge_new_data_as_delta(data, data_path, predicate):

    try:
        dt = DeltaTable(data_path)
        data_pa = pa.Table.from_pandas(data)
        dt.merge(
            source=data_pa,
            source_alias="source",
            target_alias="target",
            predicate=predicate
        ) \
        .when_matched_update_all() \
        .when_not_matched_insert_all() \
        .execute()
    except TableNotFoundError:
        save_data_as_delta(data, data_path)
    
def save_new_data_as_delta(new_data, data_path, predicate, partition_cols=None):
   
    try:
      dt = DeltaTable(data_path)
      new_data_pa = pa.Table.from_pandas(new_data)

      # Se insertan en target, datos de source que no existen en target
      dt.merge(
          source=new_data_pa,
          source_alias="source",
          target_alias="target",
          predicate=predicate
      ) \
      .when_not_matched_insert_all() \
      .execute()

    # Si no existe la tabla Delta Lake, se guarda como nueva
    except TableNotFoundError:
      save_data_as_delta(new_data, data_path, partition_cols=partition_cols)


#### Extracción Full:

In [ ]:
# Obtengo api-keys

parser = ConfigParser()
parser.read("pipeline.conf")
api_key = parser["api-credentials"]["api-key"]         

urlBase = "https://api.jcdecaux.com/vls/v1"

In [ ]:
paramsEstatico = {
        "apiKey" : api_key
    }

endpointEstatico = "contracts"

urlEstatica = f"{urlBase}/{endpointEstatico}"
    
df_estatico = utils.obtenerDatosEstaciones(urlEstatica, paramsEstatico)


#### Extracción Incremental:

In [ ]:
endpointDinamico = "stations"

paramsDinamico = {
    "contract": "lyon",
    "apiKey" : api_key
}

urlDinamica= f"{urlBase}/{endpointDinamico}"

df_dinamico = utils.extraccionIncremental(urlDinamica, "metadata/metadata.json", paramsDinamico)

#### Almacenamiento de Datos - Extracción Full

In [ ]:
contracts_dir = f"{bronze_dir}/{endpointEstatico}"

utils.merge_new_data_as_delta(df_estatico, contracts_dir, "target.name = source.name")

#### Almacenamiento de Datos - Extracción Incremental

In [ ]:
stations_dir = f"{bronze_dir}/{endpointDinamico}"

if not df_dinamico.empty:

    df_dinamico["last_update"] = utils.pd.to_datetime(df_dinamico.last_update)
    
    df_dinamico["fecha"] = df_dinamico.last_update.dt.date
    
    df_dinamico["hora"] = df_dinamico.last_update.dt.hour

    utils.save_new_data_as_delta(df_dinamico, stations_dir, predicate= "target.number = source.number AND target.last_update = source.last_update", partition_cols=["fecha", "hora"])